In [1]:
from pathlib import Path
import datetime

debug_mode = True

%load_ext autoreload
%autoreload 2

base_path = Path("/workspaces/SolarMED")
date_span=["20180921", "20180921"]
env_date_span = ["20180921", "20180928"]
evaluation_id="dev"

file_id = f"results_nNLP_op_plan_eval_at_{datetime.datetime.now():%Y%m%dT%H%M}_{evaluation_id}"
output_path = Path(base_path) / f"optimization/results/{date_span[0]}_{date_span[1]}/{file_id}.h5"
output_path.parent.mkdir(parents=True, exist_ok=True)

# main(
# date_span=date_span,       
data_path= base_path / "optimization/data"
# env_date_span=env_date_span,
# output_path=output_path,
uncertainty_factor=0.4
operation_optimization_layer=False
# )



In [ ]:
# Evaluate operation plan - startup / shutdown
import argparse
import gzip
import shutil
from typing import Literal, get_args, Optional
import copy
from dataclasses import asdict, is_dataclass
from pathlib import Path
import time
import numpy as np
import pandas as pd
from loguru import logger
import pygmo as pg
import threading
import datetime
from tqdm.auto import tqdm  # notebook compatible

from solarmed_modeling.solar_med import SolarMED
from solarmed_optimization.utils.serialization import get_fitness_history
from solarmed_optimization.problems import BaseNlpProblem
from solarmed_optimization.utils.progress import update_bar_every
from solarmed_optimization import (
    EnvironmentVariables,
    ProblemParameters,
    RealDecisionVariablesUpdatePeriod,
    InitialDecVarsValues,
    ProblemData
)
from solarmed_optimization.problems.nlp import (
    AlgoParams, 
    ProblemsEvaluationParameters,
    OperationPlanResults,
    OpPlanActionType
)
from solarmed_optimization.utils.initialization import (
    problem_initialization,
    InitialStates,
    initialize_problem_instances_nNLP
)
from solarmed_optimization.utils.evaluation import (evaluate_idle_thermal_storage,
                                                    evaluate_model)
from solarmed_optimization.utils.operation_plan import generate_update_datetimes


In [ ]:
def get_initial_states(sim_df: Optional[pd.DataFrame] = None, Tts_h: Optional[list[float]] = None, Tts_c: Optional[list[float]] = None,) -> InitialStates:
    
    if sim_df is not None:
        Tts_h = [sim_df.iloc[-1][f"Tts_h_{key}"] for key in ["t", "m", "b"]]
        Tts_c = [sim_df.iloc[-1][f"Tts_c_{key}"] for key in ["t", "m", "b"]]
    elif Tts_h is None or Tts_c is None:
        Tts_h=[90, 80, 70]
        Tts_c=[70, 60, 50]
        
    return InitialStates(Tts_h=Tts_h, Tts_c=Tts_c)

def select_best_alternative(df: pd.DataFrame, std_penalty_weight: float =1.0, worst_case_penalty_weight: float =1.0) -> tuple[int, pd.DataFrame]:
    """
    Select the best alternative based on average performance, consistency (std dev), and worst-case scenario.
    
    Parameters:
    - df: pd.DataFrame
        Rows = alternatives, Columns = scenarios
    - std_penalty_weight: float
        How much to penalize alternatives with high standard deviation
    - worst_case_penalty_weight: float
        How much to penalize based on the worst-case performance
    
    Returns:
    - best_alternative: str
        Index label of the best alternative
    """
    # Calculate metrics
    means = df.mean(axis=1)
    if len(df.columns) > 1:
        stds = df.std(axis=1)
    else:
        stds = pd.Series([0] * len(df.index), index=df.index)
    worst_cases = df.max(axis=1)
    
    # Composite score (lower is better)
    scores = means + std_penalty_weight * stds + worst_case_penalty_weight * worst_cases
    
    # Reporting
    report = pd.DataFrame({
        'Mean': means,
        'Std Dev': stds,
        'Worst Case': worst_cases,
        'Composite Score': scores
    }).sort_values('Composite Score')
    
    print("\n=== Alternative Evaluation Report ===\n")
    print(report)
    print("\n======================================\n")
    
    # Pick the best
    best_alternative = scores.idxmin()
    print(f"Selected Best Alternative: {best_alternative}")
    
    return best_alternative, report

def problem_parameters_definition(action: OpPlanActionType, initial_states: InitialStates) -> ProblemParameters:
    
    if action == "startup":
        if debug_mode:
            # Simplify the combinations to have a reduced number of them
            operation_actions = {
                # Day 1 -----------------------  # Day 2 -----------------------
                "sfts": [("startup", 2), ("shutdown", 1), ("startup", 1), ("shutdown", 1)],
                "med": [("startup", 2), ("shutdown", 1), ("startup", 1), ("shutdown", 1)],
            }
        else:
            operation_actions = {
                # Day 1 -----------------------  # Day 2 -----------------------
                "sfts": [("startup", 3), ("shutdown", 3), ("startup", 1), ("shutdown", 1)],
                "med": [("startup", 3), ("shutdown", 3), ("startup", 1), ("shutdown", 1)],
            }
    
    elif action == "shutdown":
        # Shutdown operation updates
        if debug_mode:
            operation_actions= {
                # Day 1 ---------------  # Day 2 -----------------------
                "sfts": [("shutdown", 2), ("startup", 1), ("shutdown", 1)],
                "med":  [("shutdown", 2), ("startup", 1), ("shutdown", 1)],
            }
        else:
            # Shutdown operation updates
            operation_actions= {
                # Day 1 ---------------  # Day 2 -----------------------
                "sfts": [("shutdown", 3), ("startup", 2), ("shutdown", 2)],
                "med":  [("shutdown", 3), ("startup", 2), ("shutdown", 2)],
            }
        
    else:
        raise ValueError(f"Unknown action {action}. Options are: {get_args(OpPlanActionType)}")

    return ProblemParameters(
        optim_window_time=36 * 3600,  # 1d12h
        sample_time_opt=3600,  # 1h, In NLP-operation plan just used to resample environment variables
        operation_actions=operation_actions,
        initial_states=initial_states,
        real_dec_vars_update_period=RealDecisionVariablesUpdatePeriod(),
        initial_dec_vars_values=InitialDecVarsValues(), # Defaults valid for startup, not shutdown
        on_limits_violation_policy="penalize",
    )

def build_archipielago(problems: list[BaseNlpProblem], algo_params: AlgoParams, x0: list[np.ndarray] = None, fitness0: list[float] = None) -> pg.archipelago:
    
    if x0 is not None:
        assert len(problems) == len(x0), f"Number of initial populations ({len(x0)}) should match number of problems ({len(problems)})"
        assert fitness0 is not None, "Initial fitness should be provided if initial populations are provided"
    
    archi = pg.archipelago()
    for problem_idx, problem in enumerate(problems):

        # Initialize problem instance
        prob = pg.problem(problem)
        
        # Initialize population
        pop = pg.population(prob, size=algo_params.pop_size, seed=0)
        if x0 is not None and x0[problem_idx] is not None:
            pop.set_xf(0, x0[problem_idx], [fitness0[problem_idx]])
        
        algo = pg.algorithm(getattr(pg, algo_params.algo_id)(**algo_params.params_dict))
        algo.set_verbosity( algo_params.log_verbosity )
        
        # 6. Build up archipielago
        archi.push_back(
            # Setting use_pool=True results in ever-growing memory footprint for the sub-processes
            # https://github.com/esa/pygmo2/discussions/168#discussioncomment-10269386
            pg.island(udi=pg.mp_island(use_pool=False), algo=algo, pop=pop, )
        )
        
    return archi

def evaluate_problems(problems: list[BaseNlpProblem], algo_params: AlgoParams, problems_eval_params: ProblemsEvaluationParameters, action: OpPlanActionType) -> OperationPlanResults:
    # This function should sequentially, build the archipielagos, 
    # evolve them, drop poorly performing problems and repeat until
    # the best performing problems are evolved completely
    
    def update_fitness_history(isl: pg.island, fit_his: pd.Series | None, initial: bool = False) -> pd.Series:
        
        if initial:
            fit = pd.Series(isl.get_population().champion_f[0], index=[0])
        else:
            fit = get_fitness_history(algo_params.algo_id, isl.get_algorithm() )
            fit_last = pd.Series(isl.get_population().champion_f[0], 
                                 index=[problems_eval_params.n_obj_fun_evals_per_update])
            if len(fit) == 0:
                fit = fit_last
            elif fit.index[-1] < problems_eval_params.n_obj_fun_evals_per_update:
                # Append the last fitness
                fit = pd.concat([fit, fit_last])
        
        if fit_his is None or len(fit_his) == 0:
            fit_his = fit
        else:
            fit.index = fit.index + fit_his.index[-1] +1
            fit_his = pd.concat([fit_his, fit])
            
        return fit_his
    
    date_str = list(asdict(problems[0].env_vars).values())[0].index[0].strftime("%Y%m%d")
    start_time = time.time()
    
    algo_params = AlgoParams(
        algo_id=algo_params.algo_id,
        max_n_obj_fun_evals=problems_eval_params.n_obj_fun_evals_per_update,
        pop_size=algo_params.pop_size,
        max_n_logs=algo_params.max_n_logs,
    )
    
    x = [None] * len(problems)
    fitness = [None] * len(problems)
    fitness_history = [None] * len(problems)
    droped_problem_idxs = []
    kept_problem_idxs = np.arange(len(problems))
    
    progress_bar = tqdm(
        range(problems_eval_params.n_updates), 
        desc="Candidate problems evaluation",
        leave=True, 
        position=2    # Position 1, main loop is at position 0
    )
    
    for update_idx in progress_bar:
        
        # Evaluate problems for the update
        yet_to_eval_idxs = kept_problem_idxs
        batch_idx=1
        # log_header_str = f"{date_str} | Op.Plan - {action} | Evaluation step {update_idx+1}/{problems_eval_params.n_updates} | Active problems {len(kept_problem_idxs)}/{len(problems)}"
        
        while len(yet_to_eval_idxs) > 0:
            
            batch_size = min(problems_eval_params.max_n_parallel_problems, len(yet_to_eval_idxs))
            
            progress_bar.set_postfix(
                {"Active problems": f"{len(kept_problem_idxs)}/{len(problems)}"},
                {"Batch": f"{batch_idx}/{len(yet_to_eval_idxs)//batch_size+1}"},
            )
            
            archi = build_archipielago(
                problems=[problems[idx] for idx in yet_to_eval_idxs][:batch_size], 
                algo_params=algo_params,
                x0=[x[idx] for idx in yet_to_eval_idxs[:batch_size]],
                fitness0=[fitness[idx] for idx in yet_to_eval_idxs[:batch_size]],
            )
            # logger.info(f"{log_header_str} | Initialized archipelago of problems of size {batch_size}")
            
            # Add initial fitness to fitness history
            for idx, isl in enumerate(archi):
                problem_idx = yet_to_eval_idxs[idx]
                fitness_history[problem_idx] = update_fitness_history(isl, fitness_history[problem_idx], initial=True)
            
            # start_time2 = time.time()
            archi.evolve()
            # logger.info(archi)
            archi.wait_check()
            # while archi.status == pg.evolve_status.busy:
            #     time.sleep(5)
            #     print(f"Elapsed time: {time.time() - start_time:.0f}")
                # print(f"Current evolution results | Best fitness: {pop_current.champion_f[0]}, \nbest decision vector: {pop_current.champion_x}")
            
            # Update output objects
            for idx, isl in enumerate(archi):
                problem_idx = yet_to_eval_idxs[idx]
                x[problem_idx] = isl.get_population().champion_x
                fitness[problem_idx] = isl.get_population().champion_f[0]
                fitness_history[problem_idx] = update_fitness_history(isl, fitness_history[problem_idx])
            
            yet_to_eval_idxs = yet_to_eval_idxs[batch_size:]
            # logger.info(f"{log_header_str} | Completed evolution of batch {batch_idx}/{len(yet_to_eval_idxs)//batch_size+1}!. Took {int(time.time() - start_time2):.0f} seconds") 
            batch_idx+=1
        
        # Retain only the best performing problems
        fitness_current_update = np.array(copy.deepcopy(fitness))
        fitness_current_update[droped_problem_idxs] = np.nan
        kept_problem_idxs, drop_idxs = problems_eval_params.update_problems(fitness_current_update)
        droped_problem_idxs += drop_idxs
    
    evaluation_time = int(time.time() - start_time)
    longest_problem_x_idx = np.argmax([len(x_) for x_ in x])
    len_longest_x = len(x[longest_problem_x_idx])
    # dec_vec = [ for x_, problem in zip(x, problems)] # Including integer part
    # x should be padded with nans to match the length of the longest problem
    # fitness_history should be padded with nans to match lengths
    op_plan_results = OperationPlanResults(
        date_str=date_str, # Date in YYYYMMDD format
        action=action,
        x = pd.DataFrame(
            np.array([np.pad(item, (0, len_longest_x - len(item)), constant_values=np.nan) for item in x]), 
            columns = [
                f"{var_id}_step_{step_idx:03d}"
                for var_id, num_steps in asdict(problems[longest_problem_x_idx].dec_var_updates).items() if var_id not in problems[0].dec_var_int_ids
                for step_idx in range(num_steps)
            ]
        ),
        # int_dec_vars = [problem.int_dec_vars.to_dataframe() for problem in problems],
        fitness = pd.Series(fitness),
        fitness_history = pd.concat(fitness_history, axis=1).sort_index(),
        # environment_df = problems[0].env_vars.to_dataframe(),
        evaluation_time=evaluation_time,
        algo_params=algo_params,
        problems_eval_params=problems_eval_params,
    )
    
    # logger.info(f"Completed evolution process! Took {evaluation_time/60:.0f} minutes") 
    
    return op_plan_results

def evaluate_operation_plan_layer(date: datetime.datetime, env_date_span_str: str, initial_states: InitialStates, action: OpPlanActionType, data_path: Path, uncertainty_factor: float = 0., ) -> tuple[list[OperationPlanResults], BaseNlpProblem, int]:
    
    if uncertainty_factor > 0:
        unc_factors = [uncertainty_factor, 0, -uncertainty_factor]
    else:
        unc_factors = [0]
        
    op_plan_results_list: list[OperationPlanResults] = []
    problem_params = problem_parameters_definition(action, initial_states=initial_states)
    selected_date_span = [date, date + datetime.timedelta(days=problem_params.optim_window_days)]
    problem_data = problem_initialization(
        problem_params=problem_params, 
        date_str=env_date_span_str, 
        data_path=data_path,
        selected_date_span=selected_date_span,
    )
    
    if debug_mode:
        algo_params = AlgoParams(max_n_obj_fun_evals=20,)
        problems_eval_params = ProblemsEvaluationParameters(
            drop_fraction=0.5,
            max_n_obj_fun_evals=algo_params.max_n_obj_fun_evals,
            n_obj_fun_evals_per_update=5
        )
    else:
        # Set default values for the algorithm and evaluation parameters
        algo_params = AlgoParams()
        problems_eval_params = ProblemsEvaluationParameters()
    
    progress_bar = tqdm(
        unc_factors, 
        desc=f"Op.Plan - {action}. Scenarios evaluation",
        leave=True, 
        position=1    # Position 1, main loop is at position 0
    )
    
    for scenario_idx, unc_factor in enumerate(progress_bar):
        progress_bar.set_postfix({"Uncertainty factor": f"{unc_factor:.2f}"})
        
        # Modify environment
        problem_data_copy = copy.deepcopy(problem_data)
        problem_data_copy.df["I"] = problem_data_copy.df["I"] * (1 + np.random.rand(len(problem_data.df)) * unc_factor)
        
        problems = initialize_problem_instances_nNLP(
            problem_data=problem_data,        
            store_x=False,
            store_fitness=False,
        )
        op_plan_results = evaluate_problems(
            problems,
            algo_params=algo_params,
            problems_eval_params=problems_eval_params,
            action=action,
        )
        op_plan_results.evaluate_best_problem(problems=problems, model=problem_data_copy.model)
        op_plan_results.scenario_idx = scenario_idx
        op_plan_results.problem_params = problem_data.problem_params
        op_plan_results_list.append(op_plan_results)
        
    fitness_df = pd.concat([op_plan_results.fitness for op_plan_results in op_plan_results_list], axis=1)
    # Choose best alternative
    best_alternative_idx, _ = select_best_alternative(fitness_df)
    
    return op_plan_results_list, problems[best_alternative_idx], best_alternative_idx

def evaluate_operation_optimization_layer() -> None:
    raise NotImplementedError("Operation optimization layer evaluation not implemented yet")

def initialize_simulation(env_date_span_str: str, start_date: datetime.datetime, data_path: Path) -> tuple[ProblemData, EnvironmentVariables]:
    problem_params = problem_parameters_definition(action="startup", initial_states=get_initial_states())
    selected_date_span = [start_date, start_date + datetime.timedelta(days=problem_params.optim_window_days)]

    problem_data = problem_initialization(
        problem_params=problem_params, 
        date_str=env_date_span_str, 
        data_path=data_path,
        selected_date_span=selected_date_span,
    )

    # Simulate idle system from environment start until some potential operation start
    env_vars = EnvironmentVariables.initialize_from_dataframe(
        problem_data.df, 
        # Not really needed just to estimate temperature decay 
        cost_w=problem_params.env_params.cost_w,
        cost_e=problem_params.env_params.cost_e
    )
    
    return problem_data, env_vars

# def intialize_op_plan_startup():
    
#     op_action_tuple = list(problem_params.operation_actions.values())[0][0]
#     startup_dts = generate_update_datetimes(env_vars.I, n=op_action_tuple[1], action_type=op_action_tuple[0], 
#                                             irradiance_thresholds=problem_params.irradiance_thresholds)
#     dt_span = [start_date, pd.Series(startup_dts).mean()]
#     # Update dt_span so that it matches available environment
#     dt_span[0] = max(env_vars.I.index[0], dt_span[0])
    
#     model = SolarMED(**problem_data.model.dump_instance()) # Avoid modifying the original model
#     Tts_h0, Tts_c0, sim_df = evaluate_idle_thermal_storage(model=model, dt_span=dt_span, env_vars=env_vars)
#     initial_states = get_initial_states(Tts_h=Tts_h0, Tts_c=Tts_c0)

#     return initial_states

# -----------------------------------------------------------------------------------------------------
# def main(date_span: tuple[str, str], data_path: Path, env_date_span: tuple[str, str], output_path: Path, uncertainty_factor: bool, operation_optimization_layer: bool) -> None:
logger.info(f"Evaluating nNLP-operation plan optimization for date span {date_span[0]}-{date_span[-1]}")

env_date_span_str: str = f"{env_date_span[0]}_{env_date_span[1]}"
sim_df = None

start_date = datetime.datetime.strptime(date_span[0], "%Y%m%d").replace(hour=0).astimezone(tz=datetime.timezone.utc)
end_date = datetime.datetime.strptime(date_span[1], "%Y%m%d").replace(hour=23).astimezone(tz=datetime.timezone.utc)
if date_span[0] == date_span[1]:
    all_dates = [start_date]
else:
    all_dates = list(pd.date_range(start=start_date, end=end_date, freq='D', tz='UTC'))
    
problem_data, env_vars = initialize_simulation(env_date_span_str=env_date_span_str, start_date=start_date, data_path=data_path)
model = problem_data.model

progress_bar = tqdm(all_dates, desc="SolarMED | Evaluating episode", unit="day", leave=True, position=0)

# Start the parallel thread to update the main progress bar
status_update_thread = threading.Thread(target=update_bar_every, args=[progress_bar, 20], daemon=True)
status_update_thread.start()
    
for date in progress_bar:
    progress_bar.set_postfix({"Current date": date.strftime("%Y%m%d")})
    
    env_vars = env_vars.dump_in_span(span=(date, None), return_format="series")
    
    # Evaluate operation plan - startup
    op_plan_results_list, problem, best_problem_idx = evaluate_operation_plan_layer(
        date, 
        env_date_span_str, # TODO: Replace this by a datetime span that includes the evaluation start datetime
        initial_states = get_initial_states(sim_df), 
        uncertainty_factor=uncertainty_factor, 
        action="startup",
        data_path=data_path
    )
    
    # Simulate up to start
    if sim_df is None:
        sim_df = model.to_dataframe(index=env_vars.I.index[0]) 
    _, _, sim_df = evaluate_idle_thermal_storage(
        model=model, 
        dt_span=(env_vars.I.index[0], problem.operation_span[0]), 
        env_vars=env_vars, 
        df=sim_df
    )
    
    # Evaluate operation optimization
    if not operation_optimization_layer:
        # From operation start up to shutdown evaluation
        # Get earliest potential shutdown time
        pp = problem_parameters_definition(action="shutdown", initial_states=get_initial_states(sim_df))
        op_action_tuple = list(pp.operation_actions.values())[0][0]
        shutdown_dts = generate_update_datetimes(
            env_vars.I, n=op_action_tuple[1], action_type="shutdown", 
            irradiance_thresholds=problem_data.problem_params.irradiance_thresholds
        )
        dt_span = [problem.operation_span[0], pd.Series(shutdown_dts).min()]
        dec_vars = problem.decision_vector_to_decision_variables(
            x=op_plan_results_list[0].x.iloc[best_problem_idx].values # uncertainty scenario to be used: 0
        )
        sim_df = evaluate_model(
            model=model,
            dec_vars=dec_vars.dump_in_span(span=dt_span, return_format="series"),
            env_vars=env_vars.dump_in_span(span=dt_span, return_format="series"),
            # problem.env_vars are resampled to model sample time, maybe env_vars too?
            n_evals_mod=len(problem.env_vars.dump_in_span(span=dt_span, return_format="series").I),
            df_mod=sim_df
        )
    # else:
    # int_dec_vars = problem.int_dec_vars
    # evaluate_operation_optimization_layer(
    #     x0 = [op_plan_results.x[] for op_plan_results in op_plan_results_list]
    # )
    
    # Evaluate operation plan - shutdown
    op_plan_results_list, problem, best_problem_idx = evaluate_operation_plan_layer(
        date, 
        env_date_span_str, # No debería ser necesario
        initial_states = get_initial_states(sim_df), 
        uncertainty_factor=0, 
        action="shutdown",
        data_path=data_path # No debería ser necesario
    )
    # Simulate from shutdown evaluation up to shutdown 
    
    # Simulate until start of next action
    # This is where the operation optimization layer should take over and fill the gap
    # Here just assume the prediction is perfect and we will reach the next action
    # if problem is None:
    #     problem = initialize_problem_instances_nNLP(problem_data,)[0]
        
    # Simulate until next action
    # sim_df = # evaluate_optimization_nlp(
    #     x=self.x.iloc[self.best_problem_idx].values, 
    #     problem=problems[self.best_problem_idx] if isinstance(problems, list) else problems,
    #     model=SolarMED(**model.dump_instance())
    # )


2025-04-29 18:11:45.943 | INFO     | __main__:<module>:337 - Evaluating nNLP-operation plan optimization for date span 20180921-20180921


2025-04-29 18:11:49.649 | WARNING  | solarmed_optimization.utils.initialization:problem_initialization:106 - External environment data (20220524 - 20240524) (seawater temperature and salinity) is not available for the data period (20180921 - 20180928). Using data for same month and days but from closest year 2022.
2025-04-29 18:11:49.656 | INFO     | solarmed_optimization.utils.initialization:problem_initialization:115 - Selected date span: 2018-09-21 00:00:00+00:00 - 2018-09-23 00:00:00+00:00 from 2018-09-21 05:00:00+00:00 - 2018-09-23 00:00:00+00:00


SolarMED | Evaluating episode:   0%|          | 0/1 [00:00<?, ?day/s]

2025-04-29 18:11:52.873 | WARNING  | solarmed_optimization.utils.initialization:problem_initialization:106 - External environment data (20220524 - 20240524) (seawater temperature and salinity) is not available for the data period (20180921 - 20180928). Using data for same month and days but from closest year 2022.
2025-04-29 18:11:52.879 | INFO     | solarmed_optimization.utils.initialization:problem_initialization:115 - Selected date span: 2018-09-21 00:00:00+00:00 - 2018-09-23 00:00:00+00:00 from 2018-09-21 05:00:00+00:00 - 2018-09-23 00:00:00+00:00


Op.Plan - startup:   0%|          | 0/3 [00:00<?, ?it/s]

...


   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1       -20.3006        13.7059              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1       -55.0401        15.9987              2

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      2              2        18.8002    4.07171e-05              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      3              3        18.6428     0.00105889              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      3              3        -68.598    0.000829321              1
      3              3       -55.0715      0.0314318              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      5              5         -41.82    0.000426925              1
      5              5       -30.8128        10.5122              1
      5              5        18.6427    0

Candidate problems evaluation:   0%|          | 0/4 [00:00<?, ?it/s]


   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1       -39.8039       0.762466              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      2              2       -68.6032      0.0061223              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      4              4        3.78644        0.84109              1
      5              5        3.40503       0.381409              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      3              3       -41.8989      0.0793731              1
      4              4       -45.0839        3.18503              3
      5              5       -48.3734        3.28943              3

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      4              4         18.618      0.0259066              3
      5              5       -69.5192        0.91591              3

   Gen:        Fevals:          Best:   Im

Candidate problems evaluation:   0%|          | 0/4 [00:00<?, ?it/s]


   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1        3.12603        0.15926              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1       -6.59477    5.92114e-06              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1       -68.6261      0.0289985              1

   Gen:        Fevals:          Best:   Improvement:     Mutations:
      1              1       -53.1638        14.1224              2
      2              2       -29.7505        23.1557              2
      2              2       -68.8218       0.195702              3
      3              3       -68.8501      0.0283125              1
      4              4       -53.4233       0.259502              1
      4              4         -68.88      0.0298644              1
      5              5       -29.8991       0.148643              2

   Gen:        Fevals:          Best:   Imp

In [9]:
op_plan_results_list[1].algo_params


AlgoParams(algo_id='sea', max_n_obj_fun_evals=5, max_n_logs=300, pop_size=1, params_dict={'gen': 5}, log_verbosity=1, gen=5)

In [10]:
from solarmed_optimization.problems.nlp import batch_export

batch_export(output_path, op_plan_results_list)


2025-04-29 18:15:35.845 | INFO     | solarmed_optimization.problems.nlp:export:206 - Exported results to /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.h5 / /20180921/startup/scenario_00
2025-04-29 18:15:35.866 | INFO     | solarmed_optimization.problems.nlp:export:206 - Exported results to /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.h5 / /20180921/startup/scenario_01
2025-04-29 18:15:35.884 | INFO     | solarmed_optimization.problems.nlp:export:206 - Exported results to /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.h5 / /20180921/startup/scenario_02


In [13]:
# Test importing operation plan results
from solarmed_optimization.problems.nlp import batch_export

test_path = Path("/workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz")

op_plan_results_list2 = [
	OperationPlanResults.initialize(
		input_path=test_path, # output_path.with_suffix(".gz"), 
		date_str="20180921",
		action="startup",
		scenario_idx=scenario_idx,
	)
	for scenario_idx in range(3)
]


test_path = test_path.with_stem("test")
batch_export(test_path, op_plan_results_list2)

[
	OperationPlanResults.initialize(
		input_path=test_path, # output_path.with_suffix(".gz"), 
		date_str="20180921",
		action="startup",
		scenario_idx=scenario_idx,
	)
	for scenario_idx in range(3)
]


2025-04-29 18:18:04.090 | INFO     | solarmed_optimization.problems.nlp:initialize:42 - Initialized OperationPlanResults from /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz / /20180921/startup/scenario_00
2025-04-29 18:18:04.110 | INFO     | solarmed_optimization.problems.nlp:initialize:42 - Initialized OperationPlanResults from /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz / /20180921/startup/scenario_01
2025-04-29 18:18:04.129 | INFO     | solarmed_optimization.problems.nlp:initialize:42 - Initialized OperationPlanResults from /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz / /20180921/startup/scenario_02
2025-04-29 18:18:04.147 | INFO     | solarmed_optimization.problems.nlp:export:206 - Exported results to /workspaces/SolarMED/optimization/results/20180921_20180921/test.h5 / /20180921/startup/sce

[OperationPlanResults(date_str='20180921', action='startup', x=   qsf_step_000  qsf_step_001  qsf_step_002  qsf_step_003  qsf_step_004  \
 0      5.444379      7.595538      4.811792      0.842502      5.707211   
 1      5.444379      7.595538      7.712583      7.621086      5.707211   
 2      5.444379      7.595538      7.712583      7.621086      5.707211   
 3      5.444379      7.595538      7.712583      7.621086      5.707211   
 4      5.444379      7.595538      2.465822      7.621086      5.707211   
 5      5.444379      7.595538      7.712583      7.621086      5.707211   
 6      0.000000      0.000000      0.000000      0.000000      0.000000   
 7      0.000000      0.000000      0.000000      0.000000      0.000000   
 8      0.000000      0.000000      0.000000      0.000000      0.000000   
 
    qsf_step_005  qsf_step_006  qsf_step_007  qsf_step_008  qsf_step_009  ...  \
 0       3.66077      2.917706      0.857236      2.704847      4.458903  ...   
 1       3.660

In [57]:
op_plan_results_list2[1].problem_params


In [5]:
test_path = Path("/workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz")

op_plan_results = OperationPlanResults.initialize(
    input_path=test_path, # output_path.with_suffix(".gz"), 
    date_str="20180921",
    action="startup",
    scenario_idx=0,
)
op_plan_results.results_df


2025-05-01 16:56:04.045 | INFO     | solarmed_optimization.problems.nlp:initialize:284 - Initialized OperationPlanResults from /workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz / /20180921/startup/scenario_00


,penalty,Jsf_ts,current_sample,wmed_f,Tamb,I,Tmed_c_in,qts_src_sp,qts_src,qts_dis,...,Tts_c_m,Tts_c_b,dec_var_qsf,dec_var_qts_src,dec_var_qmed_s,dec_var_qmed_f,dec_var_Tmed_s_in,dec_var_Tmed_c_out,dec_var_sfts_mode,dec_var_med_mode
2018-09-21 05:06:40+00:00,NaN,NaN,0,NaN,22.556024,0.370826,21.847777,NaN,NaN,NaN,...,59.998332,47.823374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2018-09-21 05:13:20+00:00,NaN,NaN,0,NaN,22.556024,0.370826,21.847777,NaN,NaN,NaN,...,59.998332,47.823374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2018-09-21 05:20:00+00:00,NaN,NaN,0,NaN,22.556024,0.370826,21.847777,NaN,NaN,NaN,...,59.998332,47.823374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2018-09-21 05:26:40+00:00,NaN,NaN,0,NaN,22.556024,0.370826,21.847777,NaN,NaN,NaN,...,59.998332,47.823374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2018-09-21 05:33:20+00:00,NaN,NaN,0,NaN,22.556024,0.370826,21.847777,NaN,NaN,NaN,...,59.998332,47.823374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-22 16:06:40+00:00,0.0,0.0,146,35.0,33.361392,479.981914,23.078129,0.0,0.0,0.0,...,71.014279,67.916728,6.677072,12.305018,45.848566,6.850648,66.177302,31.395391,0.0,0.0
2018-09-22 16:13:20+00:00,0.0,0.0,147,35.0,33.361392,479.981914,23.078129,0.0,0.0,0.0,...,71.014231,67.831871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-22 16:20:00+00:00,0.0,0.0,148,35.0,33.361392,479.981914,23.078129,0.0,0.0,0.0,...,71.014182,67.747221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-22 16:26:40+00:00,0.0,0.0,149,35.0,33.361392,479.981914,23.078129,0.0,0.0,0.0,...,71.014134,67.662780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# TODO: Create a OpPlanVisualizer that given a OpPlanResults object has methods
# to plot:
# - Decision variables evolution
# - Timeseries results
# - Problems fitness evolution
# - 
